### Import packages

In [1]:
from transformers import AutoImageProcessor, AutoModelForImageClassification
import torch
import numpy as np
import os
from tqdm import tqdm
from torch.utils.data import DataLoader
from tqdm import tqdm
from torchvision import transforms
from datasets import load_dataset,DatasetDict,load_from_disk
from torch.utils.data import DataLoader
import os
import random
import torch
import torch.nn.functional as F
import torch.nn as nn
import random
from torch.utils.data import Subset, DataLoader





A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/opt/anaconda3/envs/ir/lib/python3.12/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/opt/anaconda3/envs/ir/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/opt/anaconda3/envs/ir/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 701, in start
    self.io_loop.start()
  File "/opt/anacon

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/opt/anaconda3/envs/ir/lib/python3.12/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/opt/anaconda3/envs/ir/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/opt/anaconda3/envs/ir/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 701, in start
    self.io_loop.start()
  File "/opt/anacon

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.



### Set seed

In [2]:
torch.manual_seed(0)
random.seed(0)
np.random.seed(0)

### Set device

In [3]:
if torch.cuda.is_available():
    device = 'cuda'
elif torch.backends.mps.is_available():
    device = 'mps'
else:
    device = 'cpu'

### Define main path

In [4]:
main_path = '/Users/aakashagarwal/Downloads/ir_assignment2/'

In [5]:
mydataset = "evanarlian/imagenet_1k_resized_256"

### Dataset

#### Load dataset

In [6]:
ds = load_dataset(mydataset)
ds = ds.rename_column("image", "img")

ds = DatasetDict({
    'train': ds['train'],
    'test': ds['val']
})

Resolving data files:   0%|          | 0/52 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/52 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/46 [00:00<?, ?it/s]

### Model load

In [7]:
processor = AutoImageProcessor.from_pretrained("google/vit-large-patch32-384",use_fast=True)
model = AutoModelForImageClassification.from_pretrained("google/vit-large-patch32-384")
# Move the model to the selected device

### Extract features of training data

#### Feature function

#### Create features of train data or load from saved file.

In [8]:
model.eval()

transform = transforms.Compose([
        transforms.Resize((384, 384)),  # Resize images to a consistent size
        transforms.ToTensor(),           # Convert PIL images to tensors
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

# Define a function to process and extract features from each image
def extract_features(batch):
    imgs = batch['img']  # This will now be a list of images in a batch

    # Preprocess each image in the batch
    img_tensors = [transform(img).unsqueeze(0).to(device) for img in imgs]  # Add batch dimension and move to device
    img_tensors = torch.cat(img_tensors, dim=0)  # Concatenate to create a batch tensor

    inputs = {'pixel_values': img_tensors}  # Adjust for model input (assuming using Hugging Face models)

    # Forward pass through the model to get features
    with torch.no_grad():
        outputs = model(**inputs, output_hidden_states=True)

    hidden_states = outputs.hidden_states  # List of all hidden states

    # Extract the final hidden state
    final_hidden_state = hidden_states[-1]  # Shape: [batch_size, num_patches, hidden_size]
    
    # Optionally, pool the features
    pooled_features = final_hidden_state.mean(dim=1)  # Shape: [batch_size, hidden_size]
    
    # Return the features as a new column for the batch
    return {"features": pooled_features.cpu().numpy()}  # 


In [9]:

output_dir = main_path+ f"features/imagenet1k/"
os.makedirs(output_dir, exist_ok=True)
output_file = os.path.join(output_dir, "vit_large/")  # Saving in Arrow format

# Check if features already exist
if os.path.exists(output_file):
    print(f"Features file already exists at {output_file}. Loading features...")
    # Load existing features
    ds_feature = load_from_disk(output_file)
else:
    print("Features file does not exist. Extracting features...")

    # Use `map` to apply the feature extraction across the entire dataset
    ds_feature = ds.map(extract_features, batched=True, batch_size=64)

    # Save the dataset with the new features to disk
    ds_feature.save_to_disk(output_file)  # Saving in the Arrow format
    print("Features saved successfully!")

Features file already exists at /Users/aakashagarwal/Downloads/ir_assignment2/features/imagenet1k/vit_large/. Loading features...


Loading dataset from disk:   0%|          | 0/22 [00:00<?, ?it/s]

In [10]:
ds_feature
ds_feature = ds_feature.rename_column("labels", "label")

#### Collate function

In [11]:
def custom_collate(batch):
    # Define the transformations you want to apply to the images
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Resize images to a consistent size
        transforms.ToTensor(),           # Convert PIL images to tensors
    ])

    # Extracting each field from the batch
    index = [item['index'] for item in batch]
    # images = [transform(item['img']) for item in batch]  # Load and transform images
    labels = torch.tensor([item['label'] for item in batch])          # Convert labels to a tensor
    features = [torch.tensor(item['features']) for item in batch]     # Convert features to tensors
    
    # Return a dictionary with batched data
    return {
        'index': index,
        # 'img': torch.stack(images),    # Stack image tensors into a single tensor
        'label': labels,
        'features': torch.stack(features),  # Stack feature tensors into a single tensor
    }


#### Dataloader

In [12]:
# Total number of training samples
n_train = len(ds_feature['train'])

# Number of samples to take for q_loader
m = 1000  # Specify the desired number of samples for q_loader

# Randomly select m unique indices from the training dataset
q_indices = random.sample(range(n_train), m)

# Create subsets for q_loader and x_dataloader
q_subset = Subset(ds_feature['train'], q_indices)
x_subset = Subset(ds_feature['train'], list(set(range(n_train)) - set(q_indices)))

# Create DataLoader instances
x_dataloader = DataLoader(x_subset, batch_size=64, drop_last=True, collate_fn=custom_collate)
q_dataloader = DataLoader(q_subset, batch_size=64, drop_last=True, collate_fn=custom_collate)
test_dataloader = DataLoader(ds_feature['test'], batch_size=64, collate_fn=custom_collate)

### Create Hash Tables for training data

In [256]:
class phi_q_model(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(phi_q_model, self).__init__()
        
        self.network = nn.Sequential(
            nn.Linear(input_dim, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, output_dim)
        )
        
        # Apply Xavier initialization to each layer
        for layer in self.network:
            if isinstance(layer, nn.Linear):
                nn.init.xavier_uniform_(layer.weight)  # or nn.init.xavier_normal_(layer.weight)
                if layer.bias is not None:
                    nn.init.zeros_(layer.bias)  # Initialize biases to zero

    def forward(self, x):
        return self.network(x)




In [257]:
class phi_x_model(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(phi_x_model, self).__init__()
        
        self.network = nn.Sequential(
            nn.Linear(input_dim, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, output_dim)
        )
        
        # Apply Xavier initialization to each layer
        for layer in self.network:
            if isinstance(layer, nn.Linear):
                nn.init.xavier_uniform_(layer.weight)  # or nn.init.xavier_normal_(layer.weight)
                if layer.bias is not None:
                    nn.init.zeros_(layer.bias)  # Initialize biases to zero

    def forward(self, x):
        return self.network(x)


In [258]:
feature_dim = len(ds_feature['train'][0]['features'])
output_dim = 512
H = 16

In [259]:
# Step 2: Initialize the transformation networks (φ_q and φ_x)
phi_q = phi_q_model(feature_dim, output_dim).to(device)
phi_x = phi_x_model(feature_dim, output_dim).to(device)

In [260]:
print([param.is_leaf for param in phi_q.parameters()])  # Should all be True
print([param.is_leaf for param in phi_x.parameters()])  # Should all be True



[True, True, True, True, True, True, True, True, True, True]
[True, True, True, True, True, True, True, True, True, True]


### Loss

In [261]:
def cosine_similarity_loss(q_features, x_features, z_q_labels, z_x_labels, epsilon=1e-8):
    device = q_features.device
    z_q_labels = z_q_labels.to(device)
    z_x_labels = z_x_labels.to(device)

    # Calculate cosine similarity
    cos_sim = F.cosine_similarity(q_features.unsqueeze(1), x_features.unsqueeze(0), dim=2)

    # Relevance labels
    relevance_labels = (z_q_labels.unsqueeze(1) == z_x_labels.unsqueeze(0)).float()

    # Clamp values for log safety
    cos_sim = cos_sim.clamp(min=epsilon, max=1 - epsilon)

    # Compute the two terms of the loss function
    pos_loss = -relevance_labels * torch.log(cos_sim)
    neg_loss = -(1 - relevance_labels) * torch.log(1 - cos_sim)

    # Combine the losses and return the mean
    return torch.mean(pos_loss + neg_loss)


In [262]:
def collision_loss(z_q_features, z_q_labels, z_x_features, z_x_labels , W_hat):
    # Separate features and labels
    # Check if lengths of labels are equal

    # Use tanh(W z) as smooth surrogate for sign function
    z_q_proj = torch.tanh(z_q_features @ W_hat).T
    
    # Check for positive (matching labels) and negative (non-matching labels) pairs
    positive_mask = (z_q_labels == z_x_labels).float().unsqueeze(1)  # For broadcasting
    negative_mask = (z_q_labels != z_x_labels).float().unsqueeze(1)  # For broadcasting
    # print(torch.tanh(z_x_features @ W_hat).shape)
    # Calculate projections
    pos_proj = torch.tanh(z_x_features @ W_hat).T * positive_mask.T
    neg_proj = torch.tanh(z_x_features @ W_hat).T * negative_mask.T

    # Ranking loss: Encourages more overlap between query and positive than negative
    margin = 1
    pos_sim = (z_q_proj * pos_proj).sum(dim=1)
    neg_sim = (z_q_proj * neg_proj).sum(dim=1)

    # Mean margin-based ranking loss
    loss = torch.mean(torch.relu(margin + neg_sim - pos_sim))
    return loss

def fence_sitting_loss(z_x, W_hat):
    # Ensure all tensors are on the same device
    device = z_x.device
    z_x = z_x.to(device)  # Ensure z_x is on the same device
    # loss = torch.mean(torch.norm(torch.abs(torch.tanh(z_x @ W_hat).T) - 1,p=1, dim=1))
    loss = torch.mean(torch.norm(torch.abs(torch.tanh(z_x @ W_hat).T) - 1,p=2 ,dim=1)) 

    return loss

def bit_balance_loss(z_x, W_hat):
    # Ensure all tensors are on the same device
    device = z_x.device
    z_x = z_x.to(device)  # Ensure z_x is on the same device
    bit_counts = torch.sum(torch.tanh(z_x @ W_hat).T, dim=0)
    loss = torch.mean(torch.abs(bit_counts))
    return loss

def total_loss(z_q, z_x, W_hat, lambda1, lambda2, lambda3):
    # Ensure all tensors are on the same device
    device = z_x[0].device  # Using z_x_features for the device
    z_q_features, z_q_labels = z_q
    z_q_features = z_q_features.to(device)
    z_q_labels = z_q_labels.to(device)
    z_x_features, z_x_labels = z_x
    z_x_features = z_x_features.to(device)
    z_x_labels = z_x_labels.to(device)
    W_hat = W_hat.to(device)
    
    bit_balance = lambda1 * bit_balance_loss(z_x_features, W_hat)
    fence_sitting = lambda2 * fence_sitting_loss(z_x_features, W_hat)
    collision = lambda3 * collision_loss(z_q_features, z_q_labels ,z_x_features, z_x_labels, W_hat)
    loss = collision + fence_sitting + bit_balance
    
    return loss , collision , fence_sitting , bit_balance



In [263]:
def train(phi_q, phi_x, x_dataloader, q_dataloader, optimizer, num_epochs=10):
    phi_q.train()
    phi_x.train()
    
    for epoch in range(num_epochs):
        total_train_loss = 0
        # Create a progress bar for the current epoch
        progress_bar = tqdm(q_dataloader, desc=f"Epoch [{epoch + 1}/{num_epochs}]", leave=False)
        
        for q_batch in progress_bar:
            for x_batch in x_dataloader:
                query_features = q_batch['features'].to(device)
                corpus_features = x_batch['features'].to(device)

                # Get transformed features (z_q and z_x)
                z_q_features = phi_q(query_features)
                z_x_features = phi_x(corpus_features)
                z_q_labels = q_batch['label']
                z_x_labels = x_batch['label']

                loss = cosine_similarity_loss(z_q_features, z_x_features, z_q_labels, z_x_labels)

                # Backpropagation and optimization step
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                total_train_loss += loss.item()
                
                progress_bar.set_postfix({'loss': loss.item()})

        average_loss = total_train_loss / len(q_dataloader)
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {average_loss:.4f}')

        # Save the models after every epoch with epoch number in filename
        model_save_path_q = f'phi_q_epoch_{epoch + 1}.pth'
        model_save_path_x = f'phi_x_epoch_{epoch + 1}.pth'

        torch.save(phi_q.state_dict(), model_save_path_q)
        torch.save(phi_x.state_dict(), model_save_path_x)

        print(f'Model phi_q saved to {model_save_path_q}')
        print(f'Model phi_x saved to {model_save_path_x}')


In [264]:
# Step 2: Initialize the transformation networks (φ_q and φ_x)
phi_q = phi_q_model(feature_dim, output_dim).to(device)
phi_x = phi_x_model(feature_dim, output_dim).to(device)

# Step 3: Define the trainable hyperplanes (W^) with tanh smoothing
W_hat = nn.Parameter(torch.empty(output_dim, H))
nn.init.xavier_uniform_(W_hat)  #

Parameter containing:
tensor([[-0.0125, -0.0656,  0.0707,  ...,  0.0405,  0.0168,  0.0195],
        [-0.0752,  0.0534,  0.0684,  ...,  0.0280,  0.0571,  0.0042],
        [ 0.0861, -0.0516,  0.0276,  ...,  0.0236,  0.0590,  0.0955],
        ...,
        [ 0.0709, -0.0388,  0.0099,  ...,  0.0368,  0.0609, -0.0889],
        [-0.0584,  0.0705, -0.0640,  ..., -0.0118,  0.0938, -0.0915],
        [-0.1043, -0.0903, -0.0412,  ..., -0.0870,  0.0879,  0.1038]],
       requires_grad=True)

In [265]:
learning_rate = 1e-5
num_epochs = 3

In [266]:
optimizer = torch.optim.Adam(list(phi_q.parameters()) + list(phi_x.parameters()), lr=learning_rate, weight_decay=1e-4)
# train(phi_q,phi_x,x_dataloader=x_dataloader,q_dataloader=q_dataloader, optimizer=optimizer, num_epochs=1)

In [267]:
# Assuming phi_q and phi_x are your model instances
model_save_path_q = 'phi_q_model.pt'  # Define the path to save phi_q model
model_save_path_x = 'phi_x_model.pt'  # Define the path to save phi_x model

In [273]:
# Load the complete model
phi_q_loaded = torch.load(model_save_path_q)
phi_x_loaded = torch.load(model_save_path_x)
W_hat_loaded = torch.load(model_save_path_x)
# If you're using GPU, you may want to move it to the appropriate device
phi_q_loaded.to(device)
phi_x_loaded.to(device)

print('Models loaded successfully!')


Models loaded successfully!


/var/folders/9x/cvl5gvjj3v31p6kyd5_lblgh0000gn/T/ipykernel_91673/768094594.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  phi_q_loaded = torch.load(model_save_path_q)
/

In [269]:
import torch
from tqdm import tqdm

def train(phi_q, phi_x, W_hat, lambda1, lambda2, lambda3, x_dataloader, q_dataloader, optimizer, num_epochs=10):
    phi_q.train()
    phi_x.train()
    
    for epoch in range(num_epochs):
        total_train_loss = 0
        # Create a progress bar for the current epoch
        progress_bar = tqdm(q_dataloader, desc=f"Epoch [{epoch + 1}/{num_epochs}]", leave=False)
        
        for q_batch in progress_bar:
            for x_batch in x_dataloader:
                query_features = q_batch['features'].to(device)
                corpus_features = x_batch['features'].to(device)

                # Get transformed features (z_q and z_x)
                z_q_features = phi_q(query_features)
                z_x_features = phi_x(corpus_features)
                z_q_labels = q_batch['label']
                z_x_labels = x_batch['label']

                z_q = z_q_features, z_q_labels
                z_x = z_x_features, z_x_labels
                
                # Compute loss
                loss, collision, fence_sitting, bit_balance = total_loss(z_q, z_x, W_hat, lambda1=lambda1, lambda2=lambda2, lambda3=lambda3)

                # Backpropagation and optimization step
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                total_train_loss += loss.item()
                
                progress_bar.set_postfix({
                    'loss': loss.item(),
                    'balance_loss': bit_balance.item(),
                    'fence_loss': fence_sitting.item(),
                    'weak_loss': collision.item()
                })

        # Calculate the average loss for the epoch
        avg_loss = total_train_loss / len(x_dataloader)
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {avg_loss:.4f}')
        
        # Save the W_hat parameters after every epoch
        torch.save(W_hat.data, f'W_hat_parameters_epoch_{epoch + 1}.pth')


In [270]:
learning_rate = 1e-5
num_epochs = 10
lambda1= 0.01
lambda2=0.01
lambda3 = 1-lambda1-lambda2
phi_q.eval()
phi_x.eval()

In [272]:
optimizer = torch.optim.Adam([W_hat], lr=learning_rate, weight_decay=1e-4)
train(phi_q,phi_x,W_hat, lambda1, lambda2, lambda3,x_dataloader=x_dataloader,q_dataloader=q_dataloader, optimizer=optimizer, num_epochs=5)

KeyboardInterrupt: 

In [231]:
torch.save(W_hat.data, 'W_hat_parameters.pth')

In [275]:
W_hat = torch.load('W_hat_parameters.pth', map_location=torch.device(device))

/var/folders/9x/cvl5gvjj3v31p6kyd5_lblgh0000gn/T/ipykernel_91673/736595380.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  W_hat = torch.load('W_hat_parameters.pth', map

In [276]:
for i,j in model.named_parameters():
    if i == 'loss_coefficients':
        print(j)

In [277]:
import random
import torch
from tqdm import tqdm

hash_dict_list = []
L = 2  # number of hash tables
J = 14  # number of bits to randomly sample from hash length
position_list = []

# Iterate through the dataset
for i in range(L):
    hash_dict = dict()
     # Randomly select J unique positions from the range of the hash_length
    random_positions = random.sample(range(H), J)
    position_list.append(random_positions)
    
    for batch_idx, data in enumerate(tqdm(x_dataloader, desc="Processing Batches")):
        
        # Extract features and compute hash code
        feature = data['features'].to(device)
        hash_value = phi_x(feature)@W_hat
        hash_code = torch.where(hash_value > 0, 1, 0)
        
        # Convert the hash code tensor to a list of tuples for dictionary key compatibility
        hash_code_list = hash_code.tolist()

       
        # Iterate through each image and label in the batch
        for j in range(len(hash_code_list)):
            # Create a unique key from the hash code of each image
            hash_code_key = [hash_code_list[j][pos] for pos in random_positions]
            hash_code_key = tuple(hash_code_key)
            # Initialize the list if this hash code key is not yet in the dictionary
            if hash_code_key not in hash_dict:
                hash_dict[hash_code_key] = []
            
            # Append the tuple (batch index, image index, label) to the list of this hash code key
            hash_dict[hash_code_key].append([data['index'][j], data['label'][j].item()])

    print(len(hash_dict.keys()))
    hash_dict_list.append(hash_dict)
    

# You now have a list of new hash tables with sampled bits, and a list of their corresponding positions


Processing Batches:   1%|▏         | 252/20019 [00:07<10:13, 32.22it/s]


KeyboardInterrupt: 

In [53]:
position_list

[[5, 3, 1, 12, 11, 15, 13, 10, 0, 14, 9, 6],
 [5, 4, 6, 7, 13, 0, 3, 1, 10, 11, 2, 8]]

#### Number of buckets in each hash table

In [54]:
for i,j in hash_dict_list[0].items():
    print(i,j[:5])
    break

(1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1) [[0, 0, 0, (14.126840591430664, -11.465263366699219, 5.023034572601318, -29.156818389892578, 20.036251068115234, 11.69014835357666, -5.303621768951416, 0.01815047487616539, 0.18797419965267181, -19.478862762451172, 2.128295421600342, 3.465289831161499, -7.226108074188232, 15.794605255126953, -15.994659423828125, -21.155792236328125, 11.096128463745117, -13.228044509887695, -3.7666175365448, -4.208788871765137, 15.252692222595215, 21.010601043701172, -59.54191970825195, 5.462034225463867, -1.476148247718811, 2.194478750228882, 10.211012840270996, -9.753870964050293, -16.167240142822266, -14.224655151367188, -4.045254230499268, -1.3426427841186523, 17.444448471069336, -8.793740272521973, 3.334364414215088, 13.548185348510742, 1.9909882545471191, 29.32215690612793, -3.8064684867858887, 2.546656608581543, -5.175571918487549, -9.018769264221191, 1.246333122253418, -21.61476707458496, -10.377633094787598, -3.0035388469696045, 14.50636100769043, 0.55656313

### Hash codes for test data

In [173]:
# Initialize a list to store similar images for each hyperplane
similar_images_list = []

# Iterate through each hyperplane
for i in range(L):
    # Create a dictionary to store similar images for this hyperplane
    similar_images = {}
    random_positions = position_list[i]
    # Process test data
    for batch_idx, data in enumerate(tqdm(test_dataloader, desc="Processing Test Batches")):
        feature = data['features'].to(device)
        

        # Compute hash values for the test data
        hash_value = phi_x(feature)@W_hat
        test_hash_code = torch.where(hash_value > 0, 1, 0)

        # Convert the hash code tensor to a list of tuples for dictionary key compatibility
        test_hash_code_list = test_hash_code.tolist()
        
        # Iterate through each test image
        for j in range(len(test_hash_code_list)):
            # Create a unique key from (batch_idx, i, label)
            test_key = (data['index'][j], data['label'][j].item())  # Convert tensor to tuple
            test_hash_code_key = [test_hash_code_list[j][pos] for pos in random_positions]
            # Create a unique key from the test hash code
            test_hash_code_key = tuple(test_hash_code_key)

            # Check if this hash code exists in the training hash dictionary
            if test_hash_code_key in hash_dict_list[i]:
                # Retrieve the corresponding images and their labels from the training hash dictionary
                similar_images[test_key] = hash_dict_list[i][test_hash_code_key]
            else:
                similar_images[test_key] = []

    # Store the similar images found for this hyperplane
    similar_images_list.append(similar_images)

# Now similar_images_list contains keys as (batch_idx, i, label) and values as similar images


Processing Test Batches: 100%|██████████| 157/157 [00:06<00:00, 23.20it/s]


In [174]:
for i,j in similar_images_list[0].items():
    print(j[:1])
    break

[[0, 13, 3, (12.045876502990723, 5.887991428375244, -2.3138439655303955, -13.887799263000488, -1.3073891401290894, -0.28828126192092896, -2.5414435863494873, 5.622004985809326, -4.3306779861450195, 8.543529510498047, -0.6436425447463989, 4.6241936683654785, -3.518933057785034, -2.811086893081665, 8.111305236816406, -8.008687973022461, 5.416823863983154, -17.91381072998047, -0.16152937710285187, 10.656754493713379, 2.0685760974884033, 8.167309761047363, 7.578545093536377, 20.31664276123047, 10.34548568725586, 3.325634241104126, 14.272555351257324, 4.633854866027832, -15.228202819824219, 4.094171047210693, -3.8235325813293457, 3.102696657180786, -9.246621131896973, -10.13953685760498, -12.762266159057617, -2.78100848197937, -5.115204811096191, 4.400285720825195, 5.852871417999268, 3.685391902923584, 17.2583065032959, 3.853842258453369, -6.876890659332275, 1.832915186882019, 11.633397102355957, 3.9622037410736084, 10.114022254943848, -6.645434379577637, 6.145081043243408, 4.59461212158203

### Create union of all hash list

In [ ]:
# Initialize the dictionary to store similar images
similar_images_dict = {}

# Iterate through the similar_images_dict to populate similar_images_list
for i in similar_images_list:
    for j in tqdm(i.keys()):
        # Check if the key already exists in similar_images_list
        if j not in similar_images_list:
            similar_images_dict[j] = []  # Initialize the list if it doesn't exist

        similar_images_dict[j].extend(i[j])  # Deep copy of the elements

100%|██████████| 10000/10000 [00:00<00:00, 728721.79it/s]


In [ ]:
# Initialize the dictionary to store similar images
unique_images_dict = {}

# Iterate through the similar_images_dict to populate unique_images_dict
for i, j in tqdm(similar_images_dict.items()):
    if i not in unique_images_dict:
        unique_images_dict[i] = []  # Initialize the list only if the key doesn't exist

    for k in j:
        if k not in unique_images_dict[i]:  # Check if 'k' is not already in the list
            unique_images_dict[i].append(k)  # Append 'k' to the list


100%|██████████| 10000/10000 [00:08<00:00, 1223.58it/s]


In [ ]:
for k,(i,j) in enumerate(unique_images_dict.items()):
    print(j)
    if k==5:
        break

[[32417, 3], [18920, 7], [9709, 0], [14435, 8], [8503, 5], [6332, 5], [4315, 8], [8067, 7], [42321, 7], [29914, 2], [20558, 3], [37323, 7], [29332, 0], [48036, 7], [16642, 3], [42798, 8], [31412, 3], [34501, 8], [24503, 7], [44233, 8], [22671, 3], [21167, 7], [19926, 9], [49488, 8], [15042, 8], [26034, 2], [5374, 8], [1770, 2], [28916, 7], [6088, 9], [5858, 0], [18599, 8], [25369, 2], [6797, 9], [8003, 0], [3445, 2], [38844, 0], [24341, 0], [27588, 8], [32477, 8], [29785, 2], [17590, 5], [46757, 2], [25127, 3], [30645, 8], [16811, 7], [16603, 8], [6300, 2], [9092, 5], [41386, 7], [7287, 6], [15836, 7], [24162, 6], [14632, 8], [33992, 0], [45573, 8], [40796, 2], [26118, 8], [20817, 8], [24365, 0], [39837, 0], [9025, 9], [20787, 0], [40646, 2], [40303, 2], [13997, 0], [15540, 8], [28209, 9], [16243, 6], [19524, 8], [42036, 8], [16908, 9], [6438, 8], [19328, 8], [41376, 0], [25278, 3], [33795, 2], [15267, 0], [100, 6], [24846, 8], [28120, 9], [18461, 8], [44552, 0], [24132, 8], [18591, 9]

In [ ]:
# Process each key in similar_images_dict with tqdm
for key, similar_images in tqdm(unique_images_dict.items(), desc="Processing keys"):
    
    test_feature = torch.tensor(ds_feature['test'][key[0]]['features'])
    # Process each similar image for the current key with tqdm
    for i, image in tqdm(enumerate(similar_images), desc=f"Processing similar images for key {key[0]}", leave=False):
        # Assuming image is a list/tuple with at least four elements
        train_feature = torch.tensor(ds_feature['train'][image[0]]['features'])
        
        # Calculate cosine similarity and store it
        similarity_score = F.cosine_similarity(test_feature, train_feature, dim=0)
        image = list(image[:2])  # Make a copy to modify
        # Append the similarity score to the copied list
        image.append(similarity_score.item())
        
        # Update the original list in similar_images
        similar_images[i] = image[:3]
        

Processing keys: 100%|██████████| 10000/10000 [12:16<00:00, 13.59it/s]


In [ ]:
len(unique_images_dict.keys())

10000

### Evaluate

In [ ]:
# Define functions to calculate Average Precision (AP) and Precision@K
def average_precision(predictions, targets):
    relevant_indices = (targets == 1).nonzero(as_tuple=True)[0]
    if len(relevant_indices) == 0:
        return 0.0

    precisions = []
    for i, idx in enumerate(relevant_indices, start=1):
        precision_at_i = (targets[:idx + 1].sum() / (idx + 1)).item()
        precisions.append(precision_at_i)

    return sum(precisions) / len(precisions)

def precision_at_k(predictions, targets, k):
    top_k_indices = torch.argsort(predictions, descending=True)[:k]
    top_k_relevant = targets[top_k_indices].sum().item()
    return top_k_relevant / k

# Prepare lists to store metric scores
map_scores = []
precision_10_scores = []
precision_50_scores = []

# Example: Iterate over your dataset to calculate metrics
for key, value in tqdm(unique_images_dict.items(), desc="Calculating metrics", total=len(similar_images_list)):
    if len(value) > 0:
        # Sort the values by similarity score (assuming v[4] is the similarity score)
        sorted_value = sorted(value, key=lambda x: x[-1], reverse=True)[:50]
        predictions = torch.tensor([v[2] for v in sorted_value])  # Similarity scores
        targets = torch.tensor([v[1] == key[1] for v in sorted_value], dtype=torch.float32)  # Relevance labels
        
        # Calculate Mean Average Precision for the current sample
        ap = average_precision(predictions, targets)
        map_scores.append(ap)
        
        # Calculate Precision@10 and Precision@50
        p10 = precision_at_k(predictions, targets, k=10)
        p50 = precision_at_k(predictions, targets, k=50)
        precision_10_scores.append(p10)
        precision_50_scores.append(p50)

# Calculate final average metrics
mean_avg_precision = sum(map_scores) / len(map_scores) if map_scores else 0.0
precision_10 = sum(precision_10_scores) / len(precision_10_scores) if precision_10_scores else 0.0
precision_50 = sum(precision_50_scores) / len(precision_50_scores) if precision_50_scores else 0.0

# Print results
print("Mean Average Precision (mAP):", mean_avg_precision)
print("Precision@10:", precision_10)
print("Precision@50:", precision_50)


Calculating metrics: 10000it [00:01, 6623.11it/s]          

Mean Average Precision (mAP): 0.7755063350586736
Precision@10: 0.6174917491749176
Precision@50: 0.3157375737573757
